In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout,MaxPool2D

/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
img_rows, img_cols = 28, 28
num_classes = 10

def train_prep(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

def test_prep(raw):
    num_images = raw.shape[0]
    x_as_array = raw.values
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x

train_file = "data/train.csv"
test_file="data/test.csv"

In [3]:
test_data = pd.read_csv(test_file)
x_test = test_prep(test_data)

train_data = pd.read_csv(train_file)
x, y = train_prep(train_data)



model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Dropout(0.25))
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=100,
          epochs=30)

Epoch 1/30
42000/42000 [==============================] - 96s 2ms/step - loss: 0.3233 - acc: 0.8981
Epoch 2/30
42000/42000 [==============================] - 91s 2ms/step - loss: 0.0983 - acc: 0.9696
Epoch 3/30
42000/42000 [==============================] - 92s 2ms/step - loss: 0.0720 - acc: 0.9776
Epoch 4/30
42000/42000 [==============================] - 86s 2ms/step - loss: 0.0574 - acc: 0.9818
Epoch 5/30
42000/42000 [==============================] - 97s 2ms/step - loss: 0.0513 - acc: 0.9841
Epoch 6/30
42000/42000 [==============================] - 90s 2ms/step - loss: 0.0427 - acc: 0.9865
Epoch 7/30
42000/42000 [==============================] - 88s 2ms/step - loss: 0.0381 - acc: 0.9872
Epoch 8/30
42000/42000 [==============================] - 92s 2ms/step - loss: 0.0360 - acc: 0.9885
Epoch 9/30
42000/42000 [==============================] - 91s 2ms/step - loss: 0.0311 - acc: 0.9907
Epoch 10/30
42000/42000 [==============================] - 91s 2ms/step - loss: 0.0301 - acc: 0.9903

In [4]:
results = model.predict(x_test)

In [5]:
results = np.argmax(results,axis = 1)

In [6]:
results = pd.Series(results,name="Label")

In [7]:
len(results)

28000

In [8]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [9]:
submission.to_csv("keras_nn_best_score.csv",index=False)